<a href="https://colab.research.google.com/github/MakC-Ukr/AP_Project_Group134/blob/master/Week8/Antonyms_generate_paraphrases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
import requests
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("drive/My Drive/Extramarks Project/Dataset versions/Self-supervised Paraphrasing/Base-dataset.csv", index_col = 0 )
df = df.reset_index()
df = df.drop("index", axis = 1)

In [ ]:
df

,question_id,question,syllabus_id,answer,question_type
0,1785194,what is the main means of transport in the dem...,86886,water,Single Choice Question
1,1326506,what do you meant by impulsive force,417075,The impulsive force acts on a body for very sh...,Subjective Question
2,1790614,in which two categories are the cultivated cro...,672789,The cultivated crops are classified into food ...,Subjective Question
3,830706,state the name by which iraq was known in the ...,85795,p Mesopotamia p,Subjective Question
4,1311224,on combustion one mole of a straight chain alc...,418738,C 2H5OH,Single Choice Question
...,...,...,...,...,...
995,518506,discuss video conferencing,3720,Through video conferencing two or more people ...,Subjective Question
996,1839517,define polymers,744131,Polymers are very long chains made of repeatin...,Subjective Question
997,1826010,the given solid shape is made from cuboids cal...,744281,This solid shape is divided into two cuboids w...,Subjective Question
998,1840130,pick an example of natural antiseptic,744134,Turmeric,Single Choice Question


In [ ]:
# Global variable
LOADED_JSONS = {}

In [ ]:
QUES_CHANGED = 0

In [ ]:
def get_antonym(orig, load_jsons):
  if orig not in load_jsons:
    api_json = requests.get('http://api.conceptnet.io/c/en/'+orig.lower()+ "?offset=0&limit=2000").json()
    load_jsons[orig] = api_json
    # if(len(api_json['edges']) > 0):
    #   print("Got json response for " + orig + str(api_json['edges'][0]))
  else:
    api_json = load_jsons[orig]
    
  ret = -1
  for i in api_json['edges']:
    if i['rel']['@id'][3:] == 'Antonym':
      if i['start']['label'].lower() != orig.lower():
        ret = i['start']['label'].lower()
        break
      elif i['end']['label'].lower() != orig.lower():
        ret = i['end']['label'].lower()
        break
  return ret

In [ ]:
def replace_word(ques, QUES_CHANGED, LOADED_JSONS):
  adj_ls = ['JJ', 'JJR', 'JJS', 'VB', 'VBD', 'VFG', 'VBN', 'VBP', 'VBZ', 'RB', 'RBR', 'RBS']
  tokens = nltk.word_tokenize(ques)
  pos_tags = nltk.pos_tag(tokens)
  # Find all indices where there is an adjective
  all_indices = []
  for tag_ind,tag in enumerate(pos_tags):
    if tag[1] in adj_ls:
      all_indices.append(tag_ind)
  # Choose a random word to change
  rand_ind = -1
  random.shuffle(all_indices)
  antonym  = -1
  while antonym == -1 and rand_ind < len(all_indices)-1:
    rand_ind+=1
    rand_adj = tokens[all_indices[rand_ind]]
    antonym = get_antonym(rand_adj, LOADED_JSONS)
  # Insert the antonym and join the sentence
  if antonym != -1:
    tokens[all_indices[rand_ind]] = antonym
    ques = ' '.join([str(w) for w in tokens])
    QUES_CHANGED += 1
  if antonym == -1:
    return (ques, antonym, '')
  else:
    return (ques, antonym, rand_adj)

In [ ]:
generated_rows = []

for index, row in df.iterrows():
  ques = row['question']
  new_ques, antonym, orig =replace_word(ques, QUES_CHANGED, LOADED_JSONS)
  if antonym != -1:
    new_row = row.copy(deep = True)
    new_row['question'] = new_ques
    new_row['question_id'] = 2*str(row['question_id'])
    new_row['orig_word'] = orig
    new_row['antonym'] = antonym
    generated_rows.append(new_row)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
newDf = pd.DataFrame(generated_rows)
newDf.to_csv("drive/My Drive/Extramarks Project/Dataset versions/Self-supervised Paraphrasing/Generated_antonyms-base_dataset.csv")